In [1]:
%matplotlib ipympl

from experimental.beacon_sim.experiment_results_pb2 import ExperimentResult
from experimental.beacon_sim.world_map_config_pb2 import WorldMapConfig
from experimental.beacon_sim.mapped_landmarks_pb2 import MappedLandmarks
import planning.probabilistic_road_map_python as prmp

import pandas as pd
import pandas.io.formats.style

from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets
import matplotlib as mpl
import re

mpl.style.use('ggplot')

import seaborn as sns

from pathlib import Path

import numpy as np

In [2]:
results_dir = Path('/home/rrg/Desktop/david_results/')
configs_dir = Path('/home/rrg/Desktop/david_configs/')
results_files = sorted(results_dir.glob("*.pb"))

protos_by_exp = {}
configs_by_exp = {}

for f in results_files:
    with open(f, 'rb') as file_in:
        data = ExperimentResult()
        data.ParseFromString(file_in.read())
        protos_by_exp[f.stem] = data

        map_config_path = configs_dir / f.stem / data.experiment_config.map_config_path
        road_map_path = configs_dir / f.stem / data.experiment_config.road_map_path
        mapped_landmark_path = configs_dir / f.stem / data.experiment_config.ekf_state_path

        wmc = WorldMapConfig()
        wmc.ParseFromString(map_config_path.read_bytes())

        assert road_map_path.exists()
        map_proto = MappedLandmarks()
        map_proto.ParseFromString(mapped_landmark_path.read_bytes())
        configs_by_exp[f.stem] = {
            'road_map': prmp.RoadMap.from_proto_string(road_map_path.read_bytes()),
            'world_map_config': wmc,
            'mapped_landmarks': map_proto,
        }

In [9]:
def protobuf_to_df(pb):
    out = defaultdict(list)
    i = 0
    for result in pb.results:
        planner_name = pb.planner_names[result.planner_id]
        if planner_name.startswith('David'):
            m = re.match(r"^David Planner (FG)-(\d+\.\d+)",planner_name)#Edit when new variables are added
            out['planner_type'].append('David')
            out['planner_size'].append(None)
            out['favor_goal'].append(m.group(2))
        #out['num_visits'].append(...)
        #out['num_paths_saved'].append(...)

        out['elapsed_time'].append(result.elapsed_time.count / 1e9)
        out['trial_id'].append(result.trial_id)
        out['planner_id'].append(result.planner_id)
        out['planner_name'].append(planner_name)
        out['start'].append(np.array(pb.start_goal[result.trial_id].start.data))
        out['goal'].append(np.array(pb.start_goal[result.trial_id].goal.data))
        out['timeout'].append(pb.experiment_config.plan_timeout_s)
        if result.HasField('plan'):
            out['plan'].append(list(result.plan.nodes))
            out['log_prob_mass'].append(result.plan.log_prob_mass)
            out['expected_det'].append(result.plan.expected_det)
        else:
            out['plan'].append(None)
            out['log_prob_mass'].append(None)
            out['expected_det'].append(None)
    return pd.DataFrame(out)

def extract_clusters_from_potential(potential_proto):
    
    potential_case = potential_proto.WhichOneof('potential_oneof')
    if potential_case in ['correlated_beacon_potential', 'anticorrelated_potential', 'precision_matrix_potential']:
        return [list(getattr(potential_proto, potential_case).members)]  
    elif potential_case == 'combined_potential':
        out = []
        for pot in potential_proto.combined_potential.potentials:
            out.extend(extract_clusters_from_potential(pot))
        return out
    print('unknown potential type', potential_case)
    raise NotImplementedError
    ...

def plot_mapped_landmarks(mapped_landmarks, potential_proto):
    # extract clusters from potential
    clusters = extract_clusters_from_potential(potential_proto)
    
    ids = list(mapped_landmarks.beacon_ids)
    for cluster in clusters:
        xs = []
        ys = []
        for beacon_id in cluster:
            idx = ids.index(beacon_id)
            beacon_in_map = mapped_landmarks.beacon_in_local[idx]
            xs.append(beacon_in_map.data[0])
            ys.append(beacon_in_map.data[1])
            # plt.text(beacon_in_map.data[0], beacon_in_map.data[1], f't_{ids[i]}')
        plt.plot(xs, ys, 'o')

def plot_road_map(road_map):
    pts = np.stack(road_map.points())
    plt.plot(pts[:, 0], pts[:, 1], '*')
    for i, pt in enumerate(pts):
        plt.text(*pt, str(i))

    if road_map.has_start_goal():
        for idx, marker in [(road_map.START_IDX, 'rs'), (road_map.GOAL_IDX, 'y*')]:
            pt = road_map.point(idx)
            plt.plot(*pt, marker)
            

    line_segments = []
    for i in range(len(pts)):
        for j in range(i+1, len(pts)):
            if road_map.adj()[i, j] != 0:
                line_segments.append([pts[i, :], pts[j,:]])

    if road_map.has_start_goal():
        for idx in [road_map.START_IDX, road_map.GOAL_IDX]:
            pt = road_map.point(idx)
            neighbors = road_map.neighbors(idx)
            for _, neighbor_pt in neighbors:
                line_segments.append([pt, neighbor_pt])
    

    edges = mpl.collections.LineCollection(line_segments, colors=(0.6, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)

def plot_plan(road_map, execution_df):
    pts = [execution_df['start'].iloc[0]]
    pts += [road_map.point(x) for x in execution_df.plan.iloc[0] if x >=0]
    pts.append(execution_df['goal'].iloc[0])
    pts = np.vstack(pts)

    plt.plot(pts[:, 0], pts[:, 1], 'm')
    plt.plot(pts[0, 0], pts[0, 1], 'gs')
    plt.plot(pts[-1, 0], pts[-1, 1], 'rv')
    ...

        

In [10]:
sub_dfs = []
for name, proto in protos_by_exp.items():
    sub_df = protobuf_to_df(proto)
    sub_df['exp_name'] = name
    sub_df['exp_type'] = name.split('_')[0]
    sub_dfs.append(sub_df)
df = pd.concat(sub_dfs)
df.reset_index(inplace=True)

ValueError: All arrays must be of the same length

In [5]:
df

NameError: name 'df' is not defined